In [9]:
from tensorflow import keras
from keras import layers
import os
import numpy as np
import matplotlib

In [2]:
original_dim = (256,256)
intermediate_dim = 64
latent_dim = 2

In [3]:
inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')


In [4]:
# Create the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

In [11]:
from PIL import Image
subreddits = os.listdir("Data")
for subreddit in subreddits:
    subreddit_folder = os.path.join("Data", subreddit)
    image_paths = [os.path.join(subreddit_folder, filename) for filename in os.listdir(subreddit_folder) if ".csv" not in filename]
    for image_path in image_paths:
        image = Image.open(image_path)
        

        im_cropped = image.crop((left, upper,right,lower))
        print(im_cropped.size)
        plt.imshow(np.asarray(im_cropped))
        image_array = np.asarray(image).T
        

[[[175 176 173 ... 104 102 100]
  [167 168 167 ... 100  98  97]
  [163 164 162 ...  94  93  92]
  ...
  [156 152 150 ... 106 105 104]
  [148 147 148 ... 108 105 102]
  [141 143 144 ... 109 105 102]]

 [[150 151 150 ...  91  89  87]
  [142 143 144 ...  89  87  86]
  [138 139 139 ...  88  87  86]
  ...
  [131 128 125 ...  95  94  93]
  [123 122 123 ...  97  94  91]
  [116 118 121 ...  98  94  91]]

 [[172 173 170 ... 100  98  96]
  [164 165 164 ...  97  95  94]
  [159 160 159 ...  92  91  90]
  ...
  [127 126 128 ...  99  98  97]
  [126 125 127 ... 101  98  95]
  [120 122 127 ... 102  98  95]]]
